In [1]:

import re
import pickle
from collections import Counter

# Set up OpenAI API key
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

import random

# Set the seed for reproducibility
random.seed(42)


In [ ]:
from huggingface_hub import login
login("")
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import re
import pickle
from collections import Counter
from openai import OpenAI
from mistralai import Mistral
# Set up OpenAI API key
client_openai=  OpenAI(api_key="",
)

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)


tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

client_mistral = Mistral(api_key="5O")

import random

# Set the seed for reproducibility
random.seed(42)


/home/sneupane/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


# system_prompt_nepali = "तपाईं एक उत्कृष्ट भाषाविज्ञ हुनुहुन्छ। "

def generate_NER(model_name,prompt):


    generated_text=""

    if model_name=="openai":
        completion = client_openai.chat.completions.create(
            model = 'gpt-4o',
            messages = [
                # {'role':'system',"content": "You are an excellent linguist. "},
            #   {'role':'system',"content": "तपाईं एक उत्कृष्ट भाषाविज्ञ हुनुहुन्छ। "},
                {'role': 'user', 'content':prompt}
            ],
            # temperature = 0  ,
                max_tokens=500,
                temperature=0,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  n=1
            )

        generated_text = completion.choices[0].message.content.strip()

    

    return generated_text


In [4]:
entities_list=["Location","Date","Person","Organization","Event"]

In [5]:
def parse_file(file_path):
  with open(file_path,"r") as f:
    tokens,labels = [],[]
    t,l = [], []
    for line in f.readlines():
      tmp = line.strip().split()
      if len(tmp) == 0:
        tokens.append(t)
        labels.append(l)
        t, l = [], []
      else:
        t.append(tmp[0])
        l.append(tmp[1])
    if len(t) > 0:
      tokens.append(t)
      labels.append(l)
    data = tokens,labels
    return data

def get_news_data_sets():
  train_data= parse_file("everest-ner/EverestNER-train-bio.txt")
  test_data= parse_file("everest-ner/EverestNER-test-bio.txt")
  return train_data,test_data

def get_tweets_data_sets():
  train_data = parse_file("DanfeNER/DanfeNER-train-bio.txt")
  test_data = parse_file("DanfeNER/DanfeNER-test-bio.txt")
  return train_data,test_data



In [6]:
news_train,news_test=get_news_data_sets()
news_train_sentences, news_train_labels = news_train
news_test_sentences, news_test_labels = news_test
len(news_train_sentences),len(news_test_sentences)

(13848, 1950)

In [7]:
def convert_to_bio(entity, prediction,original_sentence):
    tokens = prediction.split()
    original_tokens=original_sentence.split()
    bio_labels = []
    inside_entity = False  # Tracks if we are inside an entity

    for token in tokens:
        if '@@' in token and '##' in token:  # Entire entity in one token
            bio_labels.append(f"B-{entity}")
            inside_entity = False
        elif '@@' in token:  # Entity begins in this token
            bio_labels.append(f"B-{entity}")
            inside_entity = True
        elif '##' in token:  # Entity ends in this token
            bio_labels.append(f"I-{entity}")
            inside_entity = False
        else:
            if inside_entity:  # Continuation of the entity
                bio_labels.append(f"I-{entity}")
            else:  # Outside of any entity
                bio_labels.append("O")
    if len(bio_labels)==0:
        bio_labels=["O"]*len(original_tokens)

    return bio_labels

In [8]:
def align_sentences(S, T):
    """
    Aligns the predicted sentence T to match the reference sentence S
    while preserving special tokens ('@@' and '##') in T.

    Parameters:
    S (str): The reference sentence.
    T (str): The predicted sentence.

    Returns:
    str: The aligned version of T, matching the structure of S.
    """
    # Split sentences into tokens
    s_tokens = S.split()
    t_tokens = T.split()

    aligned_t_tokens = []
    t_index = 0  # Pointer for T tokens

    for s_token in s_tokens:
        if t_index < len(t_tokens):
            t_token = t_tokens[t_index]

            # If the current token in T contains special markers, preserve it.
            if '@@' in t_token or '##' in t_token:
                aligned_t_tokens.append(t_token)
                t_index += 1  # Move to the next token in T
            else:
                # Align tokens from T to match S
                if t_token == s_token:
                    aligned_t_tokens.append(t_token)
                else:
                    aligned_t_tokens.append(s_token)
                t_index += 1
        else:
            # If T is shorter than S, pad with tokens from S
            aligned_t_tokens.append(s_token)

    # Handle any remaining tokens in T after exhausting S
    while t_index < len(t_tokens):
        aligned_t_tokens.append(t_tokens[t_index])
        t_index += 1

    # Join aligned tokens back into a sentence
    return ' '.join(aligned_t_tokens)

In [9]:
def post_process_output(output):
    prediction = output
    
    # Check if "Output:" is in the prediction and process accordingly
    if "Output:" in prediction:
        prediction = prediction.split("Output:")[1].strip()

    if "नतिजा:" in prediction:
        prediction = prediction.split("नतिजा:")[1].strip()

    if "वाक्य:" in prediction:
        prediction = prediction.split("वाक्य:")[1].strip()
    
    # Extract portion before "Note" if it exists
    if "Note" in prediction:
        prediction = prediction.split("Note", 1)[0].strip()
    
    # Extract up to the first occurrence of "।"
    if "।" in prediction:
        prediction = prediction.split("।", 1)[0] + "।"
    
    # Return the processed prediction
    return prediction

In [ ]:
with open("output_path/train_datasets_with_tagging.pkl", "rb") as file:  # "rb" stands for read binary
    train_datasets_with_tagging = pickle.load(file)

with open("output_path/test_datasets_with_tagging.pkl", "rb") as file:  # "rb" stands for read binary
    test_datasets_with_tagging = pickle.load(file)


In [11]:
ground_truth_separated={}
for sentence in test_datasets_with_tagging:
    # if sentence == "\' माइती टाढा ।":
        gd=test_datasets_with_tagging[sentence][1]
        a={}
        for ent in entities_list:
            temp=["O"]*len(gd)
            pos=[index for index, label in enumerate(gd) if ent in label]
            if len(pos)>0:
                for i in pos:
                    temp[i]=gd[i]
            # if len(temp)!=len(gd):
                # print("I")
            a[ent]=temp
        ground_truth_separated[sentence]=a

In [12]:
# ground_truth_separated

In [ ]:
output_file_name="k_10_semantic_NEP.pkl"

with open("output_path/"+output_file_name , "rb") as file:  # "rb" stands for read binary
    k_10_semantic_NEP = pickle.load(file)

In [19]:
import random

def select_random_keys(input_dict, num_keys=100, seed=42):
    """
    Selects a specified number of keys randomly from a dictionary.
    
    Args:
        input_dict (dict): The dictionary to sample keys from.
        num_keys (int): The number of keys to select.
        seed (int): The seed for random number generator.
    
    Returns:
        list: A list of randomly selected keys.
    """
    random.seed(seed)  # Set the seed for reproducibility
    all_keys = list(input_dict.keys())
    
    # Ensure the number of keys does not exceed the available keys
    if num_keys > len(all_keys):
        raise ValueError("num_keys exceeds the number of available keys in the dictionary.")
    
    return random.sample(all_keys, num_keys)

 # Example dictionary with 500 keys
output_file_name=k_10_semantic_NEP
random_keys = select_random_keys(output_file_name, num_keys=100, seed=123)

print(random_keys)

['ती सबै उपहार हरु विद्युतीय लक्कीड्र मार्फत विजेता छनौट गरिए को थियो ।', 'हिजो एकछत्र प्रयोग भए पनि अनलाइन प्रणाली मा सेयर धितोकर्जा को प्रयोग अव्यवहारिक हुन्छ ।', 'चिकित्सा शिक्षा अध्ययनका लागि वार्षिक अर्बौं रुपैयाँ विदेशिने भएका ले त्यस लाई रोक्न जरूरी रहे को र चिकित्सा विधेयक पास भए पछि त्यस्तो अवस्था को अन्त्य हुने उन ले उल्लेख गरे ।', 'केसी समूह को मुख्य आपत्ति विधेयक को बुँदा ७ ( ३ ) मा छ ।', 'पदयात्री हरु सुकेटार बाट पाथीभरा सम्म हिँडेरै ओहोर दोहोर गरेका थिए ।', 'कतार एयरवेज ग्रुप ले चिनियाँ एयरलाइन्स चाइना साउदर्न एयरलाइन्स ग्रुप को पाँच प्रतिशत सेयर स्वामित्व लिए को छ ।', 'छोरा को अहिले को हालखबर सुनाइहँदा मन्त्री मगर ले एक वर्षअघि पति विगोग हुँदा र निर्वाचन मा होमिँदा आफू लाई पीडा र कठिनाइ भएको स्मरण गरिन् ।', 'नदी भित्र पनि दिंदैनन् , कसरी प्रसारण लाइन बनाउनुः कुलमान', 'रियलक राफयल भरान रातो कार्ड खादै मैदान बाट बाहिरिए पनि त्यस को फाइदा उठाउदै इस्पान्योल ले गोल गरे पनि हार टार्न सकेनन् ।', 'विद्यार्थी बेच्ने कारखानावाला ले केसी लाई उचालिरेहका छन् : योगेश भट्टराई', 'नानीछो

In [31]:
def validate_predictions(sentence, ground_truths, predictions):
    updated_predictions = {}
    all_prompts={}
    # Iterate over each entity type in the predictions
    for entity_type, prediction in predictions.items():
        ground_truth = ground_truths.get(entity_type, [])
        updated_predictions[entity_type] = ["O"] * len(prediction)
        tokens = sentence.split()

        i = 0
        while i < len(prediction):
            if prediction[i].startswith("B-"):
                # Start of a new entity span
                entity_span = [tokens[i]]
                entity_indices = [i]
                entity_label = prediction[i][2:]  # Extract the label type

                # Collect continuation tokens
                j = i + 1
                while j < len(prediction) and prediction[j] == f"I-{entity_label}":
                    entity_span.append(tokens[j])
                    entity_indices.append(j)
                    j += 1

                # Validate the entire entity span
                entity_value = " ".join(entity_span)

                # Prompt GPT to validate
                # prompt = f"Is '{entity_value}' a {entity_type} in this sentence: '{sentence}'? Please answer with Yes or No. No explanation is needed."
                if len(entity_span)>1:
                    prompt=f"The given sentence: {sentence} \nAre the words {entity_value} in the given sentence a {entity_type} entity? Please answer with Yes or No. No explanation is needed. "
                else:
                    prompt=f"The given sentence: {sentence} \nIs the word {entity_value} in the given sentence a {entity_type} entity? Please answer with Yes or No. No explanation is needed. "
                # print(prompt)
                
                
                try:
                    answer=generate_NER("openai",prompt)
                    # print(answer)
                    all_prompts[prompt]=answer
                    # Update the prediction based on GPT's response
                    if "Yes" in answer:
                        # print("I came here")
                        for idx in entity_indices:
                            updated_predictions[entity_type][idx] = prediction[idx]
                    
                except Exception as e:
                    print(f"Error querying OpenAI: {e}")
                    # updated_predictions[entity_type].append("O")
            # else:
            #     updated_predictions[entity_type].append("O")
            i+=1
    return updated_predictions,all_prompts

In [32]:
y_true1=[]
y_pred1=[]
y_pred2=[]
count=0
error_sentences={}
count=0
self_corrections={}
original_predictions={}
new_predictions={}
for sentence in random_keys[:]:
    true_label=ground_truth_separated[sentence]
    temp={}
    temp1={}
    temp2={}
    for entity in entities_list:

        prediction=(output_file_name[sentence][entity])
        prediction=post_process_output(prediction)

        predicted_sentences=convert_to_bio(entity,prediction,sentence)
       
        if len(predicted_sentences)==len(true_label[entity]):

            y_true1.append(true_label[entity])
            y_pred1.append(predicted_sentences)
            
            validated_predictions,all_prompts = validate_predictions(sentence, {entity:true_label[entity]}, {entity:predicted_sentences})
            y_pred2.append(validated_predictions[entity])
            temp1[entity]=predicted_sentences
            temp2[entity]=validated_predictions
            temp[entity]=all_prompts
    self_corrections[sentence]=temp
    original_predictions[sentence]=temp1
    new_predictions[sentence]=temp2

In [33]:
print(classification_report(y_true1,y_pred1, zero_division=1))

              precision    recall  f1-score   support

        Date       0.50      0.71      0.59        35
       Event       0.07      1.00      0.13         2
    Location       0.42      0.88      0.57        24
Organization       0.47      0.82      0.60        40
      Person       0.78      0.93      0.85        55

   micro avg       0.50      0.85      0.63       156
   macro avg       0.45      0.87      0.55       156
weighted avg       0.58      0.85      0.67       156



In [34]:
print(classification_report(y_true1,y_pred2, zero_division=1))

              precision    recall  f1-score   support

        Date       0.58      0.20      0.30        35
       Event       0.11      1.00      0.19         2
    Location       0.58      0.88      0.70        24
Organization       0.64      0.72      0.68        40
      Person       0.93      0.93      0.93        55

   micro avg       0.66      0.71      0.68       156
   macro avg       0.57      0.75      0.56       156
weighted avg       0.71      0.71      0.68       156

